In [56]:
# 함수형 api를 활용한 모델 구성

In [57]:
from tensorflow.keras.datasets import mnist, cifar10
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D

In [58]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [59]:
x_train.shape

(60000, 28, 28)

In [60]:
x_train_rs = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test_rs = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255

In [61]:
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)

In [62]:
# 함수형 API

In [63]:
inputs = Input(shape=(28,28,1))
x = Conv2D(32, (3,3), activation='relu')(inputs)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)

model = Model(inputs = inputs, outputs = x)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_194 (Conv2D)             │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_195 (Conv2D)             │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │     2,359,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,380,938 (9.08 MB)

 Trainable params: 2,380,938 (9.08 MB)

 Non-trainable params: 0 (0.00 B)

In [64]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [65]:
# model.fit(x_train_rs, y_train_ohe,
#           batch_size=32,
#           validation_data=(x_test_rs, y_test_ohe),
#           epochs=10)

In [66]:
# ResNet Architecture => residual block

In [67]:
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, Activation, add, \
                          Activation, Flatten, Dense, GlobalAveragePooling2D, Reshape, multiply, concatenate, Dropout, multiply


In [68]:
def Residual_Block(x, filters):
  x_skip = x
  f = filters

  x = Conv2D(f, (3,3), padding='same')(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation('relu')(x)

  x = Conv2D(f, (3,3), padding='same')(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation('relu')(x)

  x_skip = Conv2D(f, (1,1), strides=(1,1,))(x_skip)
  x_skip = BatchNormalization(axis=3)(x_skip)

  x = add([x, x_skip])
  x = Activation('relu')(x)

  return x


In [69]:
def Residual_Block_50(x, filters):
  x_skip = x
  f1, f2 = filters

  x = Conv2D(f1, (1,1), padding='same')(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation('relu')(x)

  x = Conv2D(f1, (3,3), padding='same')(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation('relu')(x)

  x = Conv2D(f2, (1,1), padding='same')(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation('relu')(x)

  x_skip = Conv2D(f2, (1,1), strides=(1,1,))(x_skip)
  x_skip = BatchNormalization(axis=3)(x_skip)

  x = add([x, x_skip])
  x = Activation('relu')(x)

  return x



In [70]:
# ResNet

In [71]:
input = Input(shape = (28, 28,1))

x = Conv2D(64, (7,7), strides=(2,2), padding='same')(input)
x = MaxPooling2D((3,3), strides=(2,2), padding='same')(x)

for i in range(3):
  x = Residual_Block_50(x, [64, 256])

for i in range(4):
  x = Residual_Block_50(x, [128, 512])

for i in range(6):
  x = Residual_Block_50(x, [256, 1024])

for i in range(3):
  x = Residual_Block_50(x, [512, 2048])

x = Residual_Block(x, filters=64)
x = Residual_Block(x, filters=64)

x = Residual_Block(x, filters=128)
x = Residual_Block(x, filters=128)

x = Residual_Block(x, filters=256)
x = Residual_Block(x, filters=256)

x = Residual_Block(x, filters=512)
x = Residual_Block(x, filters=512)

x = GlobalAveragePooling2D()(x)

x = Dense(10, activation='softmax')(x)

model = Model(inputs = input, outputs = x)
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_196 (Conv2D) │ (None, 14, 14,    │      3,200 │ input_layer_10[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 7, 7, 64)  │          0 │ conv2d_196[0][0]  │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_197 (Conv2D) │ (None, 7, 7, 64)  │      4,160 │ max_pooling2d_6[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 64)  │        256 │ conv2d_197[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_176      │ (None, 7, 7, 64)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_198 (Conv2D) │ (None, 7, 7, 64)  │     36,928 │ activation_176[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 64)  │        256 │ conv2d_198[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_177      │ (None, 7, 7, 64)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_199 (Conv2D) │ (None, 7, 7, 256) │     16,640 │ activation_177[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 256) │      1,024 │ conv2d_199[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_200 (Conv2D) │ (None, 7, 7, 256) │     16,640 │ max_pooling2d_6[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_178      │ (None, 7, 7, 256) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 256) │      1,024 │ conv2d_200[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_48 (Add)        │ (None, 7, 7, 256) │          0 │ activation_178[0… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_179      │ (None, 7, 7, 256) │          0 │ add_48[0][0]      │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_201 (Conv2D) │ (None, 7, 7, 64)  │     16,448 │ activation_179[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 64)  │        256 │ conv2d_201[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 50,999,946 (194.55 MB)

 Trainable params: 50,912,906 (194.22 MB)

 Non-trainable params: 87,040 (340.00 KB)

In [72]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [73]:
# model.fit(x_train_rs, y_train_ohe,
#           batch_size=32,
#           validation_data=(x_test_rs, y_test_ohe),
#           epochs=10)

In [74]:
# GoogleNet

In [75]:
def inception_module(x):
  x1 = Conv2D(32, (1,1), activation='relu')(x)

  x2 = Conv2D(48, (1,1), activation='relu')(x)
  x2 = Conv2D(64, (3,3), activation='relu', padding='same')(x2)

  x3 = Conv2D(64, (1,1), activation='relu')(x)
  x3 = Conv2D(96, (3,3), activation='relu', padding='same')(x3)

  x4 = AveragePooling2D((3,3), strides=(1,1), padding='same')(x)
  x4 = Conv2D(32, (1,1), activation='relu')(x4)

  x = concatenate([x1, x2, x3, x4], axis=3)
  x = Activation('relu')(x)

  return x

In [76]:
def SEmodule(pre_layer, ch, r):
  x1 = GlobalAveragePooling2D()(pre_layer)
  x1 = Dense(int(ch/r), activation='relu')(x1)
  x1 = Dense(ch, activation='sigmoid')(x1)
  x1 = Reshape((1,1,ch))(x1)

  x2 = multiply([pre_layer, x1])
  return x2

In [77]:
def inception(x, filters):

  pre_layer = x
  f1, f2, f3, f4 = filters

  x1 = Conv2D(f1, (1,1), activation='relu', padding='same')(pre_layer)

  x2 = Conv2D(f4, (1,1), activation='relu', padding='same')(pre_layer)
  x2 = Conv2D(f2, (3,3), activation='relu', padding='same')(x2)

  x3 = Conv2D(f4, (1,1), activation='relu', padding='same')(pre_layer)
  x3 = Conv2D(f2, (3,3), activation='relu', padding='same')(x3)

  x4 = MaxPooling2D((3,3), strides=(1,1), padding='same')(pre_layer)
  x4 = Conv2D(f4, (1,1), activation='relu', padding='same')(x4)

  concat = concatenate([x1, x2, x3, x4], axis=-1)
  x = SEmodule(concat, f1+f2+f3+f4, 16)

  return x

In [78]:
# googlenet

In [79]:
# GoogleNet CNN Architecture

input_shape = x_train[0].shape
inputs = Input(shape=input_shape)  # inputs

x = Conv2D(64, kernel_size=(7,7), strides=2, padding='same', activation='relu')(inputs)
x = BatchNormalization()(x)
x = Conv2D(192, kernel_size=(3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)

x = inception(x, [64, 128, 32, 32])
x = inception(x, [128, 192, 96, 64])
x = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)

x = inception(x, [192, 208, 48, 64])
aux1 = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
aux1 = Conv2D(128, kernel_size=(1,1), padding='same', activation='relu')(aux1)
aux1 = Flatten()(aux1)
aux1 = Dense(512, activation='relu')(aux1)
aux1 = Dense(10, activation='softmax')(aux1)  # outputs 1

x = inception(x, [160, 224, 64, 64])

x = inception(x, [128, 256, 64, 64])

x = inception(x, [112, 288, 64, 64])
aux2 = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
aux2 = Conv2D(128, kernel_size=(1,1), padding='same', activation='relu')(aux2)
aux2 = Flatten()(aux2)
aux2 = Dense(832, activation='relu')(aux2)
aux2 = Dense(10, activation='softmax')(aux2)   # outputs 2

x = inception(x, [256, 320, 128, 128])
x = inception(x, [256, 320, 128, 128])

x = inception(x, [384, 384, 128, 128])
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Flatten()(x)

outputs = Dense(10, activation='softmax')(x)  # outputs 3

model = Model(inputs= inputs, outputs = [aux1, aux2, outputs])

model.compile(optimizer='adam', loss='categorical_crossentropy', loss_weights=[0.3, 0.3, 1.0], metrics=['acc'])



ValueError: Input 0 of layer "conv2d_285" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 28, 28)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d_2 (Conv2D)           (None, 14, 14, 64)           3200      ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)             0         ['conv2d_2[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_3 (Conv2D)           (None, 7, 7, 64)             4160      ['max_pooling2d_1[0][0]'

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train_rs, y_train_ohe,
          batch_size=32,
          validation_data=(x_test_rs, y_test_ohe),
          epochs=10)

Epoch 1/10
1875/1875 [==============================] - 168s 64ms/step - loss: 0.0282 - accuracy: 0.9914 - val_loss: 0.0426 - val_accuracy: 0.9864
Epoch 2/10
 846/1875 [============>.................] - ETA: 1:02 - loss: 0.0188 - accuracy: 0.9945